In [36]:
from joblib import load
import joblib
import pandas as pd
from datetime import datetime
import numpy as np

In [37]:
df = pd.read_excel("AKModeltesst_RE_raw export.xlsx")

In [ ]:
df.shape

In [39]:
details = df[['Domain', 'Name', 'Description', 'LinkedIn', 'Employee Estimate','Employee Growth (6 months)', 'Employee Growth (Annual)', 'Headquarters', 'Year Founded', 'Last Funding Amount', 'Last Funding Date', 'Total Funding Amount', 'Total Funding Rounds']]

In [40]:
data = df[['Employee Estimate', 'Employees on Professional Networks', 'Employee Growth (Monthly)', 'Employee Growth (Quarterly)', 'Employee Growth (6 months)', 'Employee Growth (Annual)', 'Headquarters', 'Year Founded', 'Last Funding Amount', 'Last Funding Date', 'Total Funding Amount', 'Total Funding Rounds', 'Business Model', 'Last Touch Date', 'Last Pipeline Decline Date']].copy()

In [ ]:
data

In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming 'data' is your input DataFrame for predictions

# US location check
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
data['is_us'] = data['Headquarters'].apply(lambda x: 0.5 if pd.isna(x) else int(any(state in x for state in us_states)))

# Calculate company age
current_year = datetime.now().year
data['company_age'] = current_year - data['Year Founded']

# Check for tech-enabled sectors
tech_enabled_sectors = ['Software', 'Software Enabled', 'Tech', 'Technology']
data['is_tech_enabled'] = data['Business Model'].apply(lambda x: 0 if pd.isna(x) else int(x in tech_enabled_sectors))

# Convert 'Employee Growth' columns to float
growth_columns = ['Employee Growth (Monthly)', 'Employee Growth (Quarterly)', 'Employee Growth (6 months)', 'Employee Growth (Annual)']
for col in growth_columns:
    data[col] = data[col].str.replace('%', '').astype(float)

# Process date columns
today = pd.Timestamp.now().tz_localize(None)
date_columns = ['Last Funding Date', 'Last Touch Date', 'Last Pipeline Decline Date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce').dt.tz_localize(None)
    data[f'Days Since {col}'] = (today - data[col]).dt.days

# Drop unnecessary columns
columns_to_drop = ['Headquarters', 'Year Founded', 'Business Model'] + date_columns
data = data.drop(columns=columns_to_drop)

In [43]:
loaded_model = load('model.joblib')

In [44]:
predictions = loaded_model.predict(data)
predictions += 1

In [ ]:
data

In [ ]:
predictions

In [47]:
details = details.copy()
details['score'] = predictions


In [48]:
cols = details.columns.tolist()
cols = cols[-1:] + cols[:-1]
details = details[cols]
details = details.sort_values(by='score', ascending=False)
details['score'] = details['score'].round(2)

In [49]:
# Save to Excel without index
details.to_excel("final_output.xlsx", index=False)